**用于估计幅度调制噪声包络的 DEMON 谱的 Python 实现。提供两个版本：平方律 DEMON 与希尔伯特变换 DEMON。根据具体应用场景，任一算法都可能更合适。两者都期望输入原始数据的数组以及滤波参数，并返回一个包含对输入数据包络估计值的输出数组。**


In [12]:
from scipy.signal import butter, lfilter, decimate, hilbert 
from numpy import square, sqrt, mean, abs 
from math import floor 
import librosa
import matplotlib.pyplot as plt


In [22]:
def demon_square_law(x, cutoff=100.0, high=3000, low=2000, fs=22050):
    """ 
    平方律DEMON算法
    
    :param x: 输入信号数组 (numpy.ndarray)
    :param cutoff: 截止频率 (float)
    :param high: 高频截止 (float)
    :param low: 低频截止 (float)
    :param fs: 采样频率 (float)
    :return: 包络信号 (numpy.ndarray)
    """ 
     # 检查参数是否满足带宽要求
    if (high+low)/2 <= 2*(high-low): 
        raise Exception("Error, band width exceeds pass band center frequency")
    else:
        # 带通滤波器参数
        nyq = 0.5 * fs # 信号带限频率 Hz
        # 通带限制作为信号带限的分数
        high /= nyq 
        low /= nyq
        order = 3 
        # Butterworth带通滤波器系数
        b, a = butter(order, [low, high], btype='band') 
        # 滤波信号
        x = lfilter(b, a, x)
        # 信号平方
        x = square(x) 
        # 计算抽取率
        n = int(floor(fs / (cutoff * 2))) 
        print(n)
        # 使用低通滤波器对信号进行n倍抽取
        x = decimate(x, n, ftype='fir') 
        # 信号开平方根
        x = sqrt(x) 
        # 减去均值
        x = x - mean(x) 

        return x 

In [23]:
def demon_hilbert_detector(x, cutoff=100.0, high=3000, low=2000, fs=22050): 
    """ 
    希尔伯特变换DEMON算法
    
    :param x: 输入信号数组 (numpy.ndarray)
    :param cutoff: 截止频率 (float)
    :param high: 高频截止 (float)
    :param low: 低频截止 (float)
    :param fs: 采样频率 (float)
    :return: 包络信号 (numpy.ndarray)
    """ 
    # 带通滤波器参数
    nyq = 0.5 * fs  # 信号带限频率 Hz 
    # 通带限制作为信号带限的分数
    high /= nyq 
    low /= nyq 
    order = 3 
    # Butterworth带通滤波器系数
    b, a = butter(order, [low, high], btype='band') 
    # 滤波信号
    x = lfilter(b, a, x) 
    # 信号的希尔伯特变换
    x = hilbert(x) 
    # 信号的绝对值
    x = abs(x) 
    # 计算抽取率
    n = int(floor(fs / (cutoff * 2))) 
    # 使用低通滤波器对信号进行n倍抽取
    x = decimate(x, n, ftype='fir') 

    # 信号开平方根
    x = sqrt(x) 

    # 减去均值
    x = x - mean(x) 

    return x 

In [24]:
audio_file = r'D:\数据集\shipsEar_AUDIOS\6__10_07_13_marDeCangas_Entra.wav'
waveform, sample_rate = librosa.load(audio_file, sr=22050)
demon_features = demon_square_law(waveform)

110


C:\Users\xiecaicheng\AppData\Local\Temp\ipykernel_39864\2955058880.py:34: RuntimeWarning: invalid value encountered in sqrt
  x = sqrt(x)
